In [1]:
%matplotlib inline
import pandas as pd
import datetime as dt
import re
import matplotlib.pyplot as plt

In [2]:
# import dataframe containing the raw data
pp_data = pd.read_csv("../data/pp_data_2018_raw.csv")

In [3]:
pp_data.shape

(27464, 9)

In [4]:
# confirm column headings
print("The raw data includes the following columns taken directly from r/progresspics")
print(pp_data.columns)

The raw data includes the following columns taken directly from r/progresspics
Index(['title', 'score', 'id', 'url', 'num_comments', 'created_utc', 'author',
       'permalink', 'timestamp'],
      dtype='object')


### Titles in the progress_pics subreddit are supposed to be formatted in the following manner:  
Gender/Age/Height \[Weight Before > Weight After = Total amount lost\](Time period in months) Personal Title

### Sample entry  
"M/26/6'0 \[190lbs&gt;175lbs=15lbs loss\] -2.5 years. All natural hard work."

### Issues with processing  
The formatting rules are not followed  
Some posters use metric units or include both English and metric units


## Processing the title 

In [5]:
# function to process the title column and extract the sex, age, height, and weights.  If the title is formatted
# incorrectly and the info cannot be extracted, "unknown" is returned. 

def get_stats_ver6(s):
    clean_s = s.upper()
    #print(clean_s)
    clean_list = clean_s.split("/")
    if len(clean_list) < 3:
        return "unknown", "unknown", "unknown", "unknown"
    sex = clean_list[0].replace(' ', '')
    age = clean_list[1].replace(' ', '')
    weight_list = clean_list[2].split("[")
    height = weight_list[0].replace(' ', '')
    try:
        weights = weight_list[1]
    except:
        weight_list2 = weight_list[0].split("(")
        height = weight_list2[0].replace(' ', '')
        try:
            weights = weight_list2[1]
        except:
            weights = "unknown"
    if weights == "unknown":
        try: 
            weights = clean_list[3]
        except: 
            weights = "unknown"
    return sex, age, height, weights

In [6]:
# add new columns: raw_sex, raw_age, raw_height, raw_weight
pp_data = pp_data.reindex(columns=['title', 'raw_sex', 'raw_age', 'raw_height', 'raw_weights', 'score', 'timestamp','id', 'num_comments', 'created_utc', 'author', 'permalink'])

In [7]:
# populate the new columns using the get_stats_ver6 function
pp_data[["raw_sex", "raw_age", "raw_height", "raw_weights"]] = pp_data.apply(lambda row: get_stats_ver6(row["title"]), axis=1, result_type="expand")

In [8]:
pp_data.head()

,title,raw_sex,raw_age,raw_height,raw_weights,score,timestamp,id,num_comments,created_utc,author,permalink
0,"M/22/6'3"" [290lbs &gt; 185lbs = 105lbs] (2 yea...",M,22,"6'3""",290LBS &GT; 185LBS = 105LBS] (2 YEARS) LONG TI...,246,2018-01-20 18:01:14,7ru6qv,10,1516492874,Bravo243,/r/progresspics/comments/7ru6qv/m2263_290lbs_1...
1,"F/27/5'10"" [355lbs &gt; 340lbs = 15lbs] (&gt;1...",F,27,"5'10""",355LBS &GT; 340LBS = 15LBS] (&GT;1 MONTH) ONLY...,576,2018-01-20 21:31:34,7rvdhz,31,1516505494,deemacd,/r/progresspics/comments/7rvdhz/f27510_355lbs_...
2,F/23/5’0” [260 &gt; 218 = 42lbs] (12 months) F...,F,23,5’0”,260 &GT; 218 = 42LBS] (12 MONTHS) FRIEND WAS G...,210,2018-01-20 21:37:49,7rvemg,5,1516505869,momicaj,/r/progresspics/comments/7rvemg/f2350_260_218_...
3,F/24/5’3” (160 SW &gt; 135 CW &gt; 125 GW) 25l...,F,24,5’3”,160 SW &GT; 135 CW &GT; 125 GW) 25LBS LOST SO ...,1,2018-01-20 22:29:43,7rvnzq,1,1516508983,solesky,/r/progresspics/comments/7rvnzq/f2453_160_sw_1...
4,F/48/5’6” [326 lbs &gt; 180lbs= 146 lbs lost] ...,F,48,5’6”,326 LBS &GT; 180LBS= 146 LBS LOST] (101 MONTHS...,3163,2018-01-20 22:31:15,7rvoaj,144,1516509075,Wendyjay34,/r/progresspics/comments/7rvoaj/f4856_326_lbs_...


In [9]:
pp_data.shape

(27464, 12)

In [10]:
# remove any entries that have "unknown" entered in the 4 new columns
pp_data = pp_data[pp_data["raw_height"] != 'unknown']

In [11]:
pp_data.shape

(23681, 12)

In [12]:
# remove entries where the length of the height column is greater than 7 characters...another tell of an incorrectly
# formated title 
pp_data = pp_data[pp_data['raw_height'].str.len() <= 6]

In [13]:
pp_data.shape

(21612, 12)

In [14]:
# function to process the raw_weight column to extract the starting and ending weights; if weights cannot be extracted,
# "unknown" is returned instead

def get_weights_ver2(s):
    stats = {}
    clean_s = s.upper().lstrip().replace(' ', '')
    regex = re.compile(r"^(?:\D*?)(\d+)(?:\D*?)(\d+)")
    result = regex.search(clean_s) 
    if result:
        #print((result.groups()))
        return result.group(1), result.group(2)
    else:
        return "unknown", "unknown"


In [15]:
pp_data.columns

Index(['title', 'raw_sex', 'raw_age', 'raw_height', 'raw_weights', 'score',
       'timestamp', 'id', 'num_comments', 'created_utc', 'author',
       'permalink'],
      dtype='object')

In [16]:
# add start_weight and end_weight columns
pp_data = pp_data.reindex(columns=['title', 'raw_sex', 'raw_age', 'raw_height', 'raw_weights', 'start_weight', 
                                         'end_weight','score','timestamp', 'id', 'num_comments', 'created_utc', 
                                         'author','permalink'])

In [17]:
# extract start and end weights from raw_weight, add to start_weight and end_weight columns
pp_data[["start_weight", "end_weight"]] = pp_data.apply(lambda row: get_weights_ver2(row["raw_weights"]), axis=1, result_type="expand")

In [18]:
# determine how many raw_start weights couldn't be processed
unknown = pp_data[pp_data["start_weight"]=="unknown"]
unknown.shape

(105, 14)

In [19]:
# looked at the unknowns and they are mostly people who didn't include any weights so remove them from the df 
pp_data = pp_data[pp_data['start_weight'] != 'unknown']

In [20]:
pp_data.shape

(21507, 14)

In [21]:
pp_data.head()

,title,raw_sex,raw_age,raw_height,raw_weights,start_weight,end_weight,score,timestamp,id,num_comments,created_utc,author,permalink
0,"M/22/6'3"" [290lbs &gt; 185lbs = 105lbs] (2 yea...",M,22,"6'3""",290LBS &GT; 185LBS = 105LBS] (2 YEARS) LONG TI...,290,185,246,2018-01-20 18:01:14,7ru6qv,10,1516492874,Bravo243,/r/progresspics/comments/7ru6qv/m2263_290lbs_1...
1,"F/27/5'10"" [355lbs &gt; 340lbs = 15lbs] (&gt;1...",F,27,"5'10""",355LBS &GT; 340LBS = 15LBS] (&GT;1 MONTH) ONLY...,355,340,576,2018-01-20 21:31:34,7rvdhz,31,1516505494,deemacd,/r/progresspics/comments/7rvdhz/f27510_355lbs_...
2,F/23/5’0” [260 &gt; 218 = 42lbs] (12 months) F...,F,23,5’0”,260 &GT; 218 = 42LBS] (12 MONTHS) FRIEND WAS G...,260,218,210,2018-01-20 21:37:49,7rvemg,5,1516505869,momicaj,/r/progresspics/comments/7rvemg/f2350_260_218_...
3,F/24/5’3” (160 SW &gt; 135 CW &gt; 125 GW) 25l...,F,24,5’3”,160 SW &GT; 135 CW &GT; 125 GW) 25LBS LOST SO ...,160,135,1,2018-01-20 22:29:43,7rvnzq,1,1516508983,solesky,/r/progresspics/comments/7rvnzq/f2453_160_sw_1...
4,F/48/5’6” [326 lbs &gt; 180lbs= 146 lbs lost] ...,F,48,5’6”,326 LBS &GT; 180LBS= 146 LBS LOST] (101 MONTHS...,326,180,3163,2018-01-20 22:31:15,7rvoaj,144,1516509075,Wendyjay34,/r/progresspics/comments/7rvoaj/f4856_326_lbs_...


## Cleaning the sex column

In [22]:
# function that processes the sex column and returns M, F or unknown if the sex column contains an age (2 digits) 

def clean_sex(s):
    if s.isdigit():
        return "unknown"
    try:
        if s[0] == "M" or s[0] == "F":
            return s[0]
    except IndexError:    
        return "error"   
    else:
        return s[-1]

In [23]:
# add the sex column
pp_data = pp_data.reindex(columns=['title', 'raw_sex', 'sex', 'raw_age', 'raw_height', 'raw_weights', 'start_weight', 
                                         'end_weight','score','timestamp', 'id', 'num_comments', 'created_utc', 
                                         'author','permalink'])

In [24]:
# apply clean_sex function to the df; store results in sex column
pp_data["sex"] = pp_data["raw_sex"].apply(clean_sex)

In [25]:
pp_data[(pp_data["sex"] == "M")|(pp_data["sex"] == "F")].shape

(21194, 15)

In [26]:
# seperate df on the basis of if sex is "unknown" or not - 243 are unknown meaning they are a number
sex_known = pp_data[pp_data["sex"] != 'unknown']
sex_unknown = pp_data[pp_data["sex"] == 'unknown']

In [27]:
sex_unknown.shape

(243, 15)

In [28]:
sex_unknown.columns

Index(['title', 'raw_sex', 'sex', 'raw_age', 'raw_height', 'raw_weights',
       'start_weight', 'end_weight', 'score', 'timestamp', 'id',
       'num_comments', 'created_utc', 'author', 'permalink'],
      dtype='object')

In [29]:
sex_unknown.head(2)

,title,raw_sex,sex,raw_age,raw_height,raw_weights,start_weight,end_weight,score,timestamp,id,num_comments,created_utc,author,permalink
159,27/M/6’4” [292lbs-265lbs = 27lbs] (2 months) a...,27,unknown,M,6’4”,292LBS-265LBS = 27LBS] (2 MONTHS) A LITTLE BIT...,292,265,1,2018-01-29 14:14:07,7tv62d,1,1517256847,[deleted],/r/progresspics/comments/7tv62d/27m64_292lbs26...
808,27/F/5'9 [217lbs &gt; 154lbs = 63lbs] 12 month...,27,unknown,F,5'9,217LBS &GT; 154LBS = 63LBS] 12 MONTHS TRANSFOR...,217,154,1,2018-01-23 12:44:30,7sgm3f,1,1516733070,[deleted],/r/progresspics/comments/7sgm3f/27f59_217lbs_1...


In [30]:
# when the sex column contains an age, then the age column usually contains the sex.  
# switch the raw_sex and raw_age column labels in the sex_unknown df
sex_unknown.columns = ['title', 'raw_age', 'sex', 'raw_sex', 'raw_height', 'raw_weights',
       'start_weight', 'end_weight', 'score', 'timestamp', 'id',
       'num_comments', 'created_utc', 'author', 'permalink']

In [31]:
sex_unknown.head(1)

,title,raw_age,sex,raw_sex,raw_height,raw_weights,start_weight,end_weight,score,timestamp,id,num_comments,created_utc,author,permalink
159,27/M/6’4” [292lbs-265lbs = 27lbs] (2 months) a...,27,unknown,M,6’4”,292LBS-265LBS = 27LBS] (2 MONTHS) A LITTLE BIT...,292,265,1,2018-01-29 14:14:07,7tv62d,1,1517256847,[deleted],/r/progresspics/comments/7tv62d/27m64_292lbs26...


In [32]:
# Process the "new" raw_sex column using clean_sex function
sex_unknown.loc[:, "sex"] = sex_unknown.loc[:, "raw_sex"].apply(clean_sex)

/Users/ajwright/anaconda3/envs/my_conda/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [33]:
sex_unknown.columns

Index(['title', 'raw_age', 'sex', 'raw_sex', 'raw_height', 'raw_weights',
       'start_weight', 'end_weight', 'score', 'timestamp', 'id',
       'num_comments', 'created_utc', 'author', 'permalink'],
      dtype='object')

In [34]:
pp_data.columns

Index(['title', 'raw_sex', 'sex', 'raw_age', 'raw_height', 'raw_weights',
       'start_weight', 'end_weight', 'score', 'timestamp', 'id',
       'num_comments', 'created_utc', 'author', 'permalink'],
      dtype='object')

In [35]:
# Switch the column order so that sex_unknown and pp_data match before updating 
sex_unknown = sex_unknown.reindex(columns=['title', 'raw_sex', 'sex', 'raw_age', 'raw_height', 'raw_weights',
       'start_weight', 'end_weight', 'score', 'timestamp', 'id',
       'num_comments', 'created_utc', 'author', 'permalink'])

In [36]:
# update pp_data with revised sex and age info from sex_unknown
pp_data.update(sex_unknown)

In [37]:
# include only entries that have a M or F in the clean_sex column in the pp_data df
pp_data = pp_data[(pp_data["sex"] == "M")|(pp_data["sex"] == "F")]

In [38]:
pp_data.shape

(21425, 15)

In [39]:
# convert M to 0 and F to 1 in the clean_sex column
pp_data["sex"] = pp_data['sex'].apply(lambda s: 1 if s == "F" else 0)

In [40]:
pp_data.head(2)

,title,raw_sex,sex,raw_age,raw_height,raw_weights,start_weight,end_weight,score,timestamp,id,num_comments,created_utc,author,permalink
0,"M/22/6'3"" [290lbs &gt; 185lbs = 105lbs] (2 yea...",M,0,22,"6'3""",290LBS &GT; 185LBS = 105LBS] (2 YEARS) LONG TI...,290,185,246.0,2018-01-20 18:01:14,7ru6qv,10.0,1.516493e+09,Bravo243,/r/progresspics/comments/7ru6qv/m2263_290lbs_1...
1,"F/27/5'10"" [355lbs &gt; 340lbs = 15lbs] (&gt;1...",F,1,27,"5'10""",355LBS &GT; 340LBS = 15LBS] (&GT;1 MONTH) ONLY...,355,340,576.0,2018-01-20 21:31:34,7rvdhz,31.0,1.516505e+09,deemacd,/r/progresspics/comments/7rvdhz/f27510_355lbs_...


## Cleaning the age column

In [41]:
# some entries have ages in the raw_height column - put those entreis in the age_info df, switch 
# the columns labels, then update the pp_data df with the new info. 

pattern = r"([0-9]{2})"
age_info = pp_data[(pp_data["raw_height"].str.match(pattern))&(pp_data["raw_height"].str.len() == 2)]


In [42]:
age_info.head(1)

,title,raw_sex,sex,raw_age,raw_height,raw_weights,start_weight,end_weight,score,timestamp,id,num_comments,created_utc,author,permalink
258,F/5'/26 [120 &gt; 94 = 26] (1YR) Weight traini...,F,1,5',26,"120 &GT; 94 = 26] (1YR) WEIGHT TRAINING, MINIM...",120,94,1.0,2018-01-08 13:59:26,7p17ga,1.0,1.515442e+09,[deleted],/r/progresspics/comments/7p17ga/f526_120_94_26...


In [43]:
age_info.shape

(115, 15)

In [44]:
age_info.columns

Index(['title', 'raw_sex', 'sex', 'raw_age', 'raw_height', 'raw_weights',
       'start_weight', 'end_weight', 'score', 'timestamp', 'id',
       'num_comments', 'created_utc', 'author', 'permalink'],
      dtype='object')

In [45]:
age_info.columns = ['title', 'raw_height', 'raw_sex', 'sex', 'raw_age', 'raw_weights',
       'start_weight', 'end_weight', 'score', 'timestamp', 'id',
       'num_comments', 'created_utc', 'author', 'permalink']
pp_data.update(age_info)

In [46]:
pp_data.shape

(21425, 15)

In [47]:
pp_data[pp_data["raw_age"].str.len() != 2].shape

(99, 15)

In [48]:
# filter out raw_ages that do not have extactly 2 digits.
pp_data = pp_data[pp_data["raw_age"].str.len() == 2]
pattern = r"(^[0-9]{2}?)"
pp_data = pp_data[pp_data["raw_age"].str.match(pattern)]

In [49]:
# convert age from strings to integers
pp_data["raw_age"] = pd.to_numeric(pp_data["raw_age"], errors='coerce')

In [50]:
pp_data.shape

(21322, 15)

In [51]:
pp_data.head(1)

,title,raw_sex,sex,raw_age,raw_height,raw_weights,start_weight,end_weight,score,timestamp,id,num_comments,created_utc,author,permalink
0,"M/22/6'3"" [290lbs &gt; 185lbs = 105lbs] (2 yea...",M,0,22,"6'3""",290LBS &GT; 185LBS = 105LBS] (2 YEARS) LONG TI...,290,185,246.0,2018-01-20 18:01:14,7ru6qv,10.0,1.516493e+09,Bravo243,/r/progresspics/comments/7ru6qv/m2263_290lbs_1...


## Cleaning the height column

In [52]:
# function that strips out all non digit characters from a string leaving behind just numbers

def number_height(s):
    chars = list(s)
    digit_chars = [c for c in chars if c.isdigit()]
    number_s = "".join(digit_chars)
    return number_s

In [53]:
# apply number_height function to each height entry and store the results in num_height column
pp_data["num_height"] = pp_data["raw_height"].apply(number_height)

In [54]:
# measurements in feet start with 3, 4, 5, while measurements in cm start with 1.  The function below
# converts num_height to the height in inches. 

def height_inches(s):
    ft_list = ["4", "5", "6", "7"]
    if s == '':
        return "unknown"
    elif s[0] == "1":
        return int(s) * 0.39370079 
    elif s[0] in ft_list:
        if len(s) == 1:
            return int(s) *12
        elif len(s) == 3 and s[2] == "5":
            return  (int(s[0]) * 12) + int(s[1]) + 0.5
        else:
            return (int(s[0]) * 12) + int(s[1:])
    else:
        return "unknown"


In [55]:
# apply height_inches function to each num_height entry and store the results in height_in column
pp_data["height_in"] = pp_data["num_height"].apply(height_inches)

In [56]:
# convert height_in from strings to integers
pp_data.loc[:,"height_in"] = pd.to_numeric(pp_data["height_in"], errors='coerce')

In [57]:
pp_data[(pp_data["height_in"] < 54) | (pp_data["height_in"] > 85)].shape

(44, 17)

In [58]:
# remove entries with height outliers (< 4'6'' and > 7'1')
pp_data = pp_data[(pp_data["height_in"] > 54) & (pp_data["height_in"] < 85)]

In [59]:
pp_data.shape

(21120, 17)

## Cleaning the weight columns 

In [60]:
pp_data.columns

Index(['title', 'raw_sex', 'sex', 'raw_age', 'raw_height', 'raw_weights',
       'start_weight', 'end_weight', 'score', 'timestamp', 'id',
       'num_comments', 'created_utc', 'author', 'permalink', 'num_height',
       'height_in'],
      dtype='object')

In [61]:
# create 2 columns to hold the processed weights, new_start_weight and new_end_weight
pp_data = pp_data.reindex(columns=['title', 'raw_age', 'raw_sex', 'sex', 'raw_height', 'num_height',
       'height_in', 'raw_weights','start_weight', 'new_start_weight', 'end_weight', 'new_end_weight', 
       'score', 'timestamp', 'id',
       'num_comments', 'created_utc', 'author', 'permalink', ], fill_value=0)

In [62]:
# make the start_weight and end_weight columns numeric
pp_data.loc[:,"start_weight"] = pd.to_numeric(pp_data["start_weight"], errors='coerce')
pp_data.loc[:,"end_weight"] = pd.to_numeric(pp_data["end_weight"], errors='coerce')

In [63]:
# to identify weights given in kgs, look for kg in the title using the regex pattern.  If found, process the start_weight
# and end_weight as kg - multiply weights by 2.20462 to convert to lbs and store in the new weight columns

pattern = r"=\s*\d+\s*KG"
pp_data.loc[pp_data.title.str.contains(pattern, case=False), "new_start_weight"] = pp_data.loc[pp_data.title.str.contains(pattern, case=False), "start_weight"] * 2.20462
pp_data.loc[pp_data.title.str.contains(pattern, case=False), "new_end_weight"] = pp_data.loc[pp_data.title.str.contains(pattern, case=False), "end_weight"] * 2.20462


In [64]:
# the regex pattern missed some weights in kgs so also screen for start_weight < 130 and the letters "kg" anywhere 
# in the title (along with the new weight column == 0 indicating it wasn't processed the first time).  Multiply by 
# 2.20462 eto convert to lbs and store in the new weight columns.

pp_data.loc[(pp_data.start_weight < 130) & pp_data.title.str.contains("kg", case=False) & (pp_data.new_start_weight == 0), "new_start_weight"] = pp_data.loc[(pp_data.start_weight < 130) & pp_data.title.str.contains("kg", case=False) & (pp_data.new_start_weight == 0), "start_weight"] * 2.20462
pp_data.loc[(pp_data.start_weight < 130) & pp_data.title.str.contains("kg", case=False) & (pp_data.new_end_weight == 0), "new_end_weight"] = pp_data.loc[(pp_data.start_weight < 130) & pp_data.title.str.contains("kg", case=False) & (pp_data.new_end_weight == 0), "end_weight"] * 2.20462


In [65]:
# for all entries not processed above, copy the starting weights to the new weight columns with no modifications. 
pp_data.loc[(pp_data.new_start_weight == 0), "new_start_weight"] = pp_data.loc[(pp_data.new_start_weight == 0), "start_weight"]
pp_data.loc[(pp_data.new_end_weight == 0), "new_end_weight"] = pp_data.loc[(pp_data.new_end_weight == 0), "end_weight"]

In [66]:
# trim off new_start_weight entries over 1000 lbs 
pp_data = pp_data[pp_data["new_start_weight"] <= 1000]

In [67]:
# trim off new_end_weight entries over 1000 lbs and less than 70 lbs 
pp_data = pp_data[pp_data['new_end_weight'] >= 70]
pp_data = pp_data[pp_data['new_end_weight'] < 1000]

In [68]:
pp_data.shape

(20143, 19)

In [69]:
pp_data.sample(10)

,title,raw_age,raw_sex,sex,raw_height,num_height,height_in,raw_weights,start_weight,new_start_weight,end_weight,new_end_weight,score,timestamp,id,num_comments,created_utc,author,permalink
11206,"F/24/5'1"" [179lbs &gt; 133lbs = 46lbs] (4 mont...",24,F,1,"5'1""",51,61.0,179LBS &GT; 133LBS = 46LBS] (4 MONTHS) BACK GAINS,179,179.0000,133,133.00000,30.0,2018-06-30 14:06:53,8v457n,3.0,1.530386e+09,[deleted],/r/progresspics/comments/8v457n/f2451_179lbs_1...
26453,M/19/6’1” [170&lt;190=20lbs] three years betwe...,19,M,0,6’1”,61,73.0,170&LT;190=20LBS] THREE YEARS BETWEEN THESE PH...,170,170.0000,190,190.00000,16.0,2018-12-24 22:09:45,a9budl,3.0,1.545711e+09,[deleted],/r/progresspics/comments/a9budl/m1961_17019020...
20350,F/26/5’3 [208 &gt; 163 = 45 lbs] (11 months) T...,26,F,1,5’3,53,63.0,208 &GT; 163 = 45 LBS] (11 MONTHS) THE LAST FE...,208,208.0000,163,163.00000,677.0,2018-10-05 13:43:11,9lp0ju,28.0,1.538765e+09,sherileen,/r/progresspics/comments/9lp0ju/f2653_208_163_...
6558,"M/22/5'11"" [90kgs &gt; 78kgs = 12kgs loss] (8 ...",22,M,0,"5'11""",511,71.0,90KGS &GT; 78KGS = 12KGS LOSS] (8 WEEKS) DAD B...,90,198.4158,78,171.96036,85.0,2018-04-13 12:55:44,8c16k4,4.0,1.523642e+09,chiiiya,/r/progresspics/comments/8c16k4/m22511_90kgs_7...
25347,F/30/5’6” [80kg &gt; 52kg = 28kg] moved across...,30,F,1,5’6”,56,66.0,80KG &GT; 52KG = 28KG] MOVED ACROSS THE COUNTR...,80,176.3696,52,114.64024,1.0,2018-12-08 13:53:59,a4dx53,0.0,1.544299e+09,[deleted],/r/progresspics/comments/a4dx53/f3056_80kg_52k...
3311,F/28/5’4”[265&gt;100&lt;45=55] (24 months). I ...,28,F,1,5’4”,54,64.0,265&GT;100&LT;45=55] (24 MONTHS). I LOST 100LB...,265,265.0000,100,100.00000,1103.0,2018-03-23 10:19:05,86lffp,36.0,1.521818e+09,HomeFin,/r/progresspics/comments/86lffp/f2854265100455...
9998,F/26/5’1” [242lbs &gt; 218lbs = 24lbs] 1 month...,26,F,1,5’1”,51,61.0,242LBS &GT; 218LBS = 24LBS] 1 MONTH. 31 DAYS S...,242,242.0000,218,218.00000,2.0,2018-06-16 12:20:17,8rktbc,0.0,1.529170e+09,[deleted],/r/progresspics/comments/8rktbc/f2651_242lbs_2...
7071,"M/24/5'10"" [154lbs to 187lbs = 33lbs] (1 year;...",24,M,0,"5'10""",510,70.0,154LBS TO 187LBS = 33LBS] (1 YEAR; 5 MONTHS) T...,154,154.0000,187,187.00000,235.0,2018-04-25 13:57:18,8ew3mq,13.0,1.524683e+09,[deleted],/r/progresspics/comments/8ew3mq/m24510_154lbs_...
8236,"23/f/5'10"" [120 &lt; 155]",23,F,1,"5'10""",510,70.0,120 &LT; 155],120,120.0000,155,155.00000,1.0,2018-05-29 18:05:22,8n3m4h,1.0,1.527635e+09,[deleted],/r/progresspics/comments/8n3m4h/23f510_120_155/
5144,"F/26/5'7"" [208lbs &gt; 165lbs = 43lbs] (1 Year...",26,F,1,"5'7""",57,67.0,208LBS &GT; 165LBS = 43LBS] (1 YEAR) STRUGGLED...,208,208.0000,165,165.00000,513.0,2018-03-07 19:15:20,82tmv3,19.0,1.520472e+09,xsakawaka,/r/progresspics/comments/82tmv3/f2657_208lbs_1...


## Calculating weight gained/lost ("weight_diff" column)

In [70]:
# populate weight_diff column by subtracting new_end_weight from new_start_weight
pp_data["weight_diff"] = pp_data["new_start_weight"] - pp_data["new_end_weight"]

In [71]:
pp_data.columns

Index(['title', 'raw_age', 'raw_sex', 'sex', 'raw_height', 'num_height',
       'height_in', 'raw_weights', 'start_weight', 'new_start_weight',
       'end_weight', 'new_end_weight', 'score', 'timestamp', 'id',
       'num_comments', 'created_utc', 'author', 'permalink', 'weight_diff'],
      dtype='object')

In [72]:
# change column order
pp_data = pp_data.reindex(columns=['title', 'raw_age', 'raw_sex', 'sex', 'raw_height', 'num_height',
       'height_in', 'raw_weights', 'start_weight', 'new_start_weight',
       'end_weight', 'new_end_weight', 'weight_diff', 'score', 'timestamp', 'id',
       'num_comments', 'created_utc', 'author', 'permalink'])

In [73]:
# any outliers to discard?  Yes the top one is referring to a breast enhancement
pp_data.sort_values("weight_diff", ascending=True)

,title,raw_age,raw_sex,sex,raw_height,num_height,height_in,raw_weights,start_weight,new_start_weight,end_weight,new_end_weight,weight_diff,score,timestamp,id,num_comments,created_utc,author,permalink
23953,"F/20/5’5” [140.655 &gt; 143.7 = 3.7lbs) - NSV,...",20,F,1,5’5”,55,65.0,"140.655 &GT; 143.7 = 3.7LBS) - NSV, I GAINED 3...",140,140.00000,655,655.0000,-515.00000,1.0,2018-11-21 16:17:12,9z7z9j,1.0,1.542839e+09,[deleted],/r/progresspics/comments/9z7z9j/f2055_140655_1...
5313,"M/32/6'1"" [138kg 305lbs &gt;84kg 185lbs = 54kg...",32,M,0,"6'1""",61,73.0,138KG 305LBS &GT;84KG 185LBS = 54KG 120LBS] 1....,138,304.23756,305,672.4091,-368.17154,387.0,2018-04-08 03:40:05,8aorzw,38.0,1.523177e+09,[deleted],/r/progresspics/comments/8aorzw/m3261_138kg_30...
21778,"M/45/6'4"" [117kg(260lbs) &gt; 94kg(208lbs) = 2...",45,M,0,"6'4""",64,76.0,117KG(260LBS) &GT; 94KG(208LBS) = 23KG(52LBS)]...,117,257.94054,260,573.2012,-315.26066,81.0,2018-10-25 04:12:08,9r8mhh,4.0,1.540459e+09,[deleted],/r/progresspics/comments/9r8mhh/m4564_117kg260...
23494,M/31/6’5/286.601lbs&gt;197.9751=90.3895,31,M,0,6’5,65,77.0,286.601LBS&GT;197.9751=90.3895,286,286.00000,601,601.0000,-315.00000,1.0,2018-11-08 20:47:00,9vgoyh,1.0,1.541732e+09,[deleted],/r/progresspics/comments/9vgoyh/m3165286601lbs...
881,"M/19/6/3"" [310lbs &gt; 180lbs = 130lbs] (1.5 Y...",19,M,0,6,6,72.0,"3"" [310LBS &GT; 180LBS = 130LBS] (1.5 YEARS) P...",3,3.00000,310,310.0000,-307.00000,105.0,2018-01-29 21:35:44,7ty62h,13.0,1.517283e+09,SHxProdigy,/r/progresspics/comments/7ty62h/m1963_310lbs_1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20367,M/31/5’11” [600lb &gt; 260lb = 340 lost] 2 1/2...,31,M,0,5’11”,511,71.0,600LB &GT; 260LB = 340 LOST] 2 1,600,600.00000,260,260.0000,340.00000,1156.0,2018-10-11 09:36:43,9naikq,128.0,1.539269e+09,Seribus99,/r/progresspics/comments/9naikq/m31511_600lb_2...
5395,"M/49/5'6"" [575?&gt;235=340 ish](Around 36 mont...",49,M,0,"5'6""",56,66.0,575?&GT;235=340 ISH](AROUND 36 MONTHS) NOT FIN...,575,575.00000,235,235.0000,340.00000,719.0,2018-04-15 06:54:25,8ceo78,64.0,1.523793e+09,ricmac68,/r/progresspics/comments/8ceo78/m4956_57523534...
13644,M/28/5'2'' [653lbs&gt;304lbs=349lbs] (9 months...,28,M,0,5'2'',52,62.0,653LBS&GT;304LBS=349LBS] (9 MONTHS) - LOVING T...,653,653.00000,304,304.0000,349.00000,0.0,2018-07-14 20:21:13,8yy53b,2.0,1.531618e+09,orbdesire,/r/progresspics/comments/8yy53b/m2852_653lbs30...
1475,M/42/5'9 [678 &gt;178=500] 3 years Follow up s...,42,M,0,5'9,59,69.0,678 &GT;178=500] 3 YEARS FOLLOW UP STORY IN IN...,678,678.00000,178,178.0000,500.00000,1.0,2018-02-02 19:50:06,7uwdx2,1.0,1.517623e+09,markdothlynn,/r/progresspics/comments/7uwdx2/m4259_678_1785...


In [74]:
ascending = pp_data.sort_values("weight_diff", ascending=True)

In [75]:
# lots of problems here...some are joke posts, some are caused by incorrect formatting, some are caused by listing
# weights in lbs and kgs
# looks like all the entries where weight_diff >= 90 are mostly correct so remove all entries with a weight_diff 
# less than that. 

ascending.head(100)

,title,raw_age,raw_sex,sex,raw_height,num_height,height_in,raw_weights,start_weight,new_start_weight,end_weight,new_end_weight,weight_diff,score,timestamp,id,num_comments,created_utc,author,permalink
23953,"F/20/5’5” [140.655 &gt; 143.7 = 3.7lbs) - NSV,...",20,F,1,5’5”,55,65.0,"140.655 &GT; 143.7 = 3.7LBS) - NSV, I GAINED 3...",140,140.00000,655,655.0000,-515.00000,1.0,2018-11-21 16:17:12,9z7z9j,1.0,1.542839e+09,[deleted],/r/progresspics/comments/9z7z9j/f2055_140655_1...
5313,"M/32/6'1"" [138kg 305lbs &gt;84kg 185lbs = 54kg...",32,M,0,"6'1""",61,73.0,138KG 305LBS &GT;84KG 185LBS = 54KG 120LBS] 1....,138,304.23756,305,672.4091,-368.17154,387.0,2018-04-08 03:40:05,8aorzw,38.0,1.523177e+09,[deleted],/r/progresspics/comments/8aorzw/m3261_138kg_30...
21778,"M/45/6'4"" [117kg(260lbs) &gt; 94kg(208lbs) = 2...",45,M,0,"6'4""",64,76.0,117KG(260LBS) &GT; 94KG(208LBS) = 23KG(52LBS)]...,117,257.94054,260,573.2012,-315.26066,81.0,2018-10-25 04:12:08,9r8mhh,4.0,1.540459e+09,[deleted],/r/progresspics/comments/9r8mhh/m4564_117kg260...
23494,M/31/6’5/286.601lbs&gt;197.9751=90.3895,31,M,0,6’5,65,77.0,286.601LBS&GT;197.9751=90.3895,286,286.00000,601,601.0000,-315.00000,1.0,2018-11-08 20:47:00,9vgoyh,1.0,1.541732e+09,[deleted],/r/progresspics/comments/9vgoyh/m3165286601lbs...
881,"M/19/6/3"" [310lbs &gt; 180lbs = 130lbs] (1.5 Y...",19,M,0,6,6,72.0,"3"" [310LBS &GT; 180LBS = 130LBS] (1.5 YEARS) P...",3,3.00000,310,310.0000,-307.00000,105.0,2018-01-29 21:35:44,7ty62h,13.0,1.517283e+09,SHxProdigy,/r/progresspics/comments/7ty62h/m1963_310lbs_1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21903,F/21/5’6 [130 &gt; 208 &gt; 155 = 78lbs gained...,21,F,1,5’6,56,66.0,"130 &GT; 208 &GT; 155 = 78LBS GAINED, 53LBS LO...",130,130.00000,208,208.0000,-78.00000,28.0,2018-10-17 20:40:16,9p4rxc,5.0,1.539827e+09,[deleted],/r/progresspics/comments/9p4rxc/f2156_130_208_...
3382,"M/15/6`2"" [105lbs &gt; 183lbs = 78lbs] (1 Year...",15,M,0,"6`2""",62,74.0,105LBS &GT; 183LBS = 78LBS] (1 YEAR) WEIGHT GA...,105,105.00000,183,183.0000,-78.00000,1.0,2018-03-31 04:55:16,88hoz9,0.0,1.522490e+09,[deleted],/r/progresspics/comments/88hoz9/m1562_105lbs_1...
5830,"M/15/6`2"" [105lbs &gt; 183lbs = 78lbs] 1 Year ...",15,M,0,"6`2""",62,74.0,105LBS &GT; 183LBS = 78LBS] 1 YEAR NATURAL TRA...,105,105.00000,183,183.0000,-78.00000,0.0,2018-04-01 02:17:43,88p6f8,1.0,1.522567e+09,sergeybagdasaryan,/r/progresspics/comments/88p6f8/m1562_105lbs_1...
17476,M/22/5’10” [103lbs&gt;180lbs=77lbs] Five years...,22,M,0,5’10”,510,70.0,103LBS&GT;180LBS=77LBS] FIVE YEARS OF LIFTING ...,103,103.00000,180,180.0000,-77.00000,89.0,2018-08-24 22:44:49,9a3xyd,13.0,1.535169e+09,_Uncas_,/r/progresspics/comments/9a3xyd/m22510_103lbs1...


In [76]:
# remove outliers 
pp_data = pp_data[pp_data['weight_diff'] >= -90]
pp_data = pp_data[pp_data['weight_diff'] < 599]

In [77]:
pp_data.shape

(20065, 20)

## Adding date time columns 

In [78]:
# change timestamp from object to datetime
pp_data["timestamp"] = pd.to_datetime(pp_data["timestamp"]) 

In [79]:
# create columns to hold different date and time metrics 
pp_data["month"] = pp_data["timestamp"].dt.month
pp_data["year"] = pp_data["timestamp"].dt.year
pp_data["day"] = pp_data["timestamp"].dt.day
pp_data["dayofweek"] = pp_data["timestamp"].dt.dayofweek
pp_data["date"] = pp_data["timestamp"].dt.date
pp_data["time"] = pp_data["timestamp"].dt.time

In [80]:
pp_data.columns

Index(['title', 'raw_age', 'raw_sex', 'sex', 'raw_height', 'num_height',
       'height_in', 'raw_weights', 'start_weight', 'new_start_weight',
       'end_weight', 'new_end_weight', 'weight_diff', 'score', 'timestamp',
       'id', 'num_comments', 'created_utc', 'author', 'permalink', 'month',
       'year', 'day', 'dayofweek', 'date', 'time'],
      dtype='object')

## Extract not safe for work

In [81]:
# function that recognizes the acronym "NSFW" in a string and returns a 1 if NSFW is present and a 0 if it is not
# NSFW stands for Not Safe For Work

def nsfw(s):
    upper_s = s.upper()
    if "NSFW" in upper_s:
        return 1
    else:
        return 0

In [82]:
# populate new NSFW column using the nsfw function
pp_data["NSFW"] = pp_data["title"].apply(nsfw)

## Multiple postings

In [83]:
# populate the new num_posts column with the number of posts in the df by the author of each entry
pp_data["num_posts"] = pp_data.groupby(['author'])['sex'].transform('count')

In [84]:
# acwb77 has the most posts
pp_data[pp_data["author"] == 'acwb77'].shape

(19, 28)

In [85]:
# however acwb77 is not at the top of the sorted df because of the 6805 posts authored by "[deleted]"
pp_data.sort_values("num_posts", ascending=False).head(2)

,title,raw_age,raw_sex,sex,raw_height,num_height,height_in,raw_weights,start_weight,new_start_weight,...,author,permalink,month,year,day,dayofweek,date,time,NSFW,num_posts
13663,F/33/5’2” [300 &gt; 255 = 45lbs] and counting....,33,F,1,5’2”,52,62.0,300 &GT; 255 = 45LBS] AND COUNTING. MORE VEGET...,300,300.0,...,[deleted],/r/progresspics/comments/8vz2c2/f3352_300_255_...,7,2018,4,2,2018-07-04,00:16:21,0,6805
13194,F/25/5’7” [213 &gt; 185 = 28lbs] (45 days),25,F,1,5’7”,57,67.0,213 &GT; 185 = 28LBS] (45 DAYS),213,213.0,...,[deleted],/r/progresspics/comments/91ou90/f2557_213_185_...,7,2018,24,1,2018-07-24,23:55:17,0,6805


In [86]:
# for every entry where the author = "[deleted]', change the num_posts to 1. 
author = pp_data[pp_data["author"] == "[deleted]"]
author.loc[:, "num_posts"] = 1
print(author.shape)
author.head(1)

(6805, 28)


,title,raw_age,raw_sex,sex,raw_height,num_height,height_in,raw_weights,start_weight,new_start_weight,...,author,permalink,month,year,day,dayofweek,date,time,NSFW,num_posts
21,F/23/5'6 [188 to 159=30 lbs lost over about a ...,23,F,1,5'6,56,66.0,188 TO 159=30 LBS LOST OVER ABOUT A YEAR] I'VE...,188,188.0,...,[deleted],/r/progresspics/comments/7runrc/f2356_188_to_1...,1,2018,20,5,2018-01-20,19:21:08,0,1


In [87]:
pp_data.update(author)

In [88]:
# fixed, now the highest number of posts is 19 again
pp_data.sort_values("num_posts", ascending=False).head(1)

,title,raw_age,raw_sex,sex,raw_height,num_height,height_in,raw_weights,start_weight,new_start_weight,...,author,permalink,month,year,day,dayofweek,date,time,NSFW,num_posts
17992,"M/40/6'1"" [542&gt;511=31] week 4 (+1) Last wee...",40.0,M,0,"6'1""",61,73.0,542&GT;511=31] WEEK 4 (+1) LAST WEEK WAS TOUGH...,542.0,542.0,...,acwb77,/r/progresspics/comments/9cnra6/m4061_54251131...,9.0,2018.0,3.0,0.0,2018-09-03,13:11:26,0.0,19.0


## Save processed dataframe to a cvs file

In [89]:
# reorder columns 
pp_data = pp_data.reindex(columns=['title', 'raw_age', 'sex', 'height_in', 'new_start_weight', 
                                   'new_end_weight', 'weight_diff', 'NSFW', 'num_posts', 
                                   'score', 'num_comments', 'timestamp', 'id', 'created_utc', 
                                   'author', 'permalink', 'month', 'year', 'day', 'dayofweek', 
                                   'date', 'time', 'raw_sex', 'raw_height', 'num_height', 
                                   'raw_weights', 'start_weight', 'end_weight'])

In [90]:
# rename columns
pp_data.columns = (['title', 'age', 'sex', 'height_in', 'start_weight', 'end_weight', 
                    'weight_diff', 'NSFW', 'num_posts', 'score', 'num_comments', 'timestamp', 
                    'id', 'created_utc', 'author', 'permalink', 'month', 'year', 'day', 
                    'dayofweek', 'date', 'time', 'raw_sex', 'raw_height', 'num_height', 
                    'raw_weights', 'raw_start_weight', 'raw_end_weight'])

In [91]:
# remove all intermediate and starting columns leaving only those that contain ready to use features
pp_data_pro = pp_data.reindex(columns=['age', 'sex', 'height_in', 'start_weight', 'end_weight','weight_diff', 'score', 
                                       'num_comments', 'month', 'dayofweek', 'NSFW', "num_posts"])

In [92]:
# save df to file
pp_data_pro.to_csv("pp_data_2018_processed.csv", index=False)

## Extract time frame of weight change

In [93]:
# identify post titles that contain a unit of measure, create a filter
units_of_measure = ['days', 'day', 'weeks', 'week', 'months', 'months', 'year', 'years']
duration = pp_data["title"].apply(lambda sentence: any(word in sentence for word in units_of_measure))

In [94]:
# create the pp_duration df which only includes rows whose title contains a unit of measure
pp_duration = pp_data[duration]

In [95]:
pp_duration.shape

(14497, 28)

In [96]:
# the below functions identify a unit of measure preceeded by a number then transform the duration into weeks
# or months.  Titles with a unit of measure, but no preceding number, return "unknown"

def get_duration_weeks(s):
    clean_s = s.lower().replace(' ', '')
    regex1 = re.compile(r"(\d+)(day|week|month|year)")
    regex2 = re.compile(r"(\d\.\d+)(day|week|month|year)")
    result2 = regex2.search(clean_s)  
    try: 
        duration2 = result2.group(1)
    except:
        result = regex1.search(clean_s)
        try:
            duration = result.group(1)
        except:
            return "unknown"
        unit = result.group(2)
        return duration_in_weeks(duration, unit)
    
    unit2 = result2.group(2)
    return duration_in_weeks(duration2, unit2)

def duration_in_weeks(period, unit):
    if unit.lower()[0] == 'd':
        return float(period)/7
    elif unit.lower()[0] == 'w':
        return float(period)
    elif unit.lower()[0] == "m":
        return float(period) * 4
    elif unit.lower()[0] == 'y':
        return float(period) * 52
    else:
        return "unknown"  
    
def get_duration_months(s):
    clean_s = s.lower().replace(' ', '')
    regex1 = re.compile(r"(\d+)(day|week|month|year)")
    regex2 = re.compile(r"(\d\.\d+)(day|week|month|year)")
    result2 = regex2.search(clean_s)  
    try: 
        duration2 = result2.group(1)
    except:
        result = regex1.search(clean_s)
        try:
            duration = result.group(1)
        except:
            return "unknown"
        unit = result.group(2)
        return duration_in_months(duration, unit)
    
    unit2 = result2.group(2)
    return duration_in_months(duration2, unit2)

def duration_in_months(period, unit):
    if unit.lower()[0] == "d":
        return float(period)/30
    elif unit.lower()[0] == 'w':
        return float(period) * 4
    elif unit.lower()[0] == "m":
        return float(period)
    elif unit.lower()[0] == 'y':
        return float(period) * 12
    else:
        return "unknown"     

In [97]:
pp_duration.columns

Index(['title', 'age', 'sex', 'height_in', 'start_weight', 'end_weight',
       'weight_diff', 'NSFW', 'num_posts', 'score', 'num_comments',
       'timestamp', 'id', 'created_utc', 'author', 'permalink', 'month',
       'year', 'day', 'dayofweek', 'date', 'time', 'raw_sex', 'raw_height',
       'num_height', 'raw_weights', 'raw_start_weight', 'raw_end_weight'],
      dtype='object')

In [98]:
# add new columns to hold the duration of weight change:  period_weeks and period_months
pp_duration = pp_duration.reindex(columns=['title', 'age', 'sex', 'height_in', 'start_weight', 'end_weight',
       'weight_diff', 'period_weeks', 'period_months','NSFW', 'num_posts', 'score', 'num_comments','timestamp', 'id',
       'created_utc', 'author', 'permalink', 'month', 'year', 'day',
       'dayofweek', 'date', 'time', 'raw_sex', 'raw_height', 'num_height',
       'raw_weights', 'raw_start_weight', 'raw_end_weight'])

In [99]:
# use the functions above to populate period_weeks and period_months columns
pp_duration['period_weeks'] = pp_duration["title"].apply(lambda s: get_duration_weeks(s))
pp_duration['period_months'] = pp_duration["title"].apply(lambda s: get_duration_months(s))

In [100]:
# examine rows where "unknown" was returned to see why
unknown_duration = pp_duration[pp_duration["period_weeks"] == "unknown"]

# titles that have a unit of time in them, but no number right before tend to mention time in a different context 
# or use words or the the word "a" to communicate the length of time.  

In [101]:
unknown_duration.shape

(1834, 30)

In [102]:
unknown_duration.sample(10)

,title,age,sex,height_in,start_weight,end_weight,weight_diff,period_weeks,period_months,NSFW,...,day,dayofweek,date,time,raw_sex,raw_height,num_height,raw_weights,raw_start_weight,raw_end_weight
14524,M/27/175cm [140kg&gt;72kg=68kg] A journey that...,27.0,0,68.897638,308.6468,158.73264,149.91416,unknown,unknown,0.0,...,21.0,5.0,2018-07-21,18:32:10,M,175CM,175,140KG&GT;72KG=68KG] A JOURNEY THAT TOOK ME ABO...,140.0,72.0
10188,"M/41/5.7"" [300lbs &gt; 204lbs = 95lbs] (6-mont...",41.0,0,67.000000,300.0000,204.00000,96.00000,unknown,unknown,0.0,...,28.0,3.0,2018-06-28,07:38:04,M,"5.7""",57,300LBS &GT; 204LBS = 95LBS] (6-MONTHS) STILL G...,300.0,204.0
24154,"F/23/5'4"" [313LBS&gt;249LBS=64LBS] (4 and a ha...",23.0,1,64.000000,313.0000,249.00000,64.00000,unknown,unknown,0.0,...,5.0,0.0,2018-11-05,21:24:49,F,"5'4""",54,313LBS&GT;249LBS=64LBS] (4 AND A HALF MONTHS) ...,313.0,249.0
10682,"M/19/6'1"" [130kg &gt; 124kg] face gains yet? T...",19.0,0,73.000000,130.0000,124.00000,6.00000,unknown,unknown,0.0,...,28.0,3.0,2018-06-28,19:53:52,M,"6'1""",61,130KG &GT; 124KG] FACE GAINS YET? THREE MONTHS.,130.0,124.0
26134,F/29/5’9” [started: 303 lbs. currently 170lbs]...,29.0,1,69.000000,303.0000,170.00000,133.00000,unknown,unknown,0.0,...,28.0,4.0,2018-12-28,06:09:31,F,5’9”,59,STARTED: 303 LBS. CURRENTLY 170LBS]. BEFORE TR...,303.0,170.0
22746,M/46/5'10 [241 lbs &gt;218 =23 lbs ] Twelve we...,46.0,0,70.000000,241.0000,218.00000,23.00000,unknown,unknown,0.0,...,27.0,5.0,2018-10-27,14:29:52,M,5'10,510,241 LBS &GT;218 =23 LBS ] TWELVE WEEK TRANSFOR...,241.0,218.0
14231,"M/23/5'11"" [270lbs &gt; 185lbs = 95lbs] Lost m...",23.0,0,71.000000,270.0000,185.00000,85.00000,unknown,unknown,0.0,...,2.0,0.0,2018-07-02,22:19:03,M,"5'11""",511,270LBS &GT; 185LBS = 95LBS] LOST MORE THAN JUS...,270.0,185.0
14094,"M/22/5;10""[229 &gt; 176 = 53] (One year today)",22.0,0,70.000000,229.0000,176.00000,53.00000,unknown,unknown,0.0,...,13.0,4.0,2018-07-13,14:57:42,M,"5;10""",510,229 &GT; 176 = 53] (ONE YEAR TODAY),229.0,176.0
26089,F/22/5’10” [ 236 &gt; 211 = 25 ] Still a ways ...,22.0,1,70.000000,236.0000,211.00000,25.00000,unknown,unknown,0.0,...,1.0,5.0,2018-12-01,11:40:09,F,5’10”,510,236 &GT; 211 = 25 ] STILL A WAYS TO GO BUT I’...,236.0,211.0
5122,M/18/5’9 [110lbs &gt; 130lbs = 20lbs] (Not eve...,18.0,0,69.000000,110.0000,130.00000,-20.00000,unknown,unknown,0.0,...,26.0,0.0,2018-03-26,08:01:08,M,5’9,59,110LBS &GT; 130LBS = 20LBS] (NOT EVEN A YEARS)...,110.0,130.0


In [103]:
# remove "unknown" duration entries
pp_duration = pp_duration[pp_duration["period_weeks"] != "unknown"]

In [104]:
pp_duration.shape

(12663, 30)

In [105]:
pd.set_option('display.max_colwidth', -1)

In [106]:
# check to make sure most of the duration df was processed correctly
pp_duration[["title", "period_weeks"]].sample(30)

,title,period_weeks
10138,"M/22/5'10"" [200+ lbs &gt; 165 lbs = 35 lbs] Approaching my goal weight after 6 months! The number on the scale is different but the biggest changes have all been mental",24
8938,"F/21/5'2"" [158lbs &gt; 128lbs = 30lbs] (1 year) Face gains! Feeling so much more confident now.",52
7221,"M/18/5’10” [202 &gt; 175 = 27] (5 months) Only started taking things seriously about 2 months ago, Lowkey jealous of other people’s results. Was pretty depressed and getting active really helped me get out of it.",20
6673,"M/31/5'8""[286 &gt; 158 = 128 lbs]( 7 months) I really have nothing to say I'm just happy and need to start going to the gym",28
869,"M / 25 / 5’11 / 220lbs &gt; 170lbs (50lbs, 8 months) Keeping it off is complicated, but doable",32
2329,M/22/6'3 [257lbs &gt; 229lbs = 28lbs lost] (3 months) Think I'm starting to see the results pay off! I'll report back in 3 months!,12
20180,F/25/5'3 [136 &gt; 118 = 18] (5 months) Forever wearing mismatched underwear but feeling my healthiest in years! Even my belly button changed shape!,20
7083,"F/24/5'2"" [220 lbs &gt; 140 lbs = -80 lbs](22 months) Sorry I've posted the first before picture already, but I don't have many before pictures. Hit 80 lbs lost. Haven't lost much in a few months but lifting has made a big difference in body measurements and how I feel overall.",88
22957,F/33/5’7”[176lbs&gt;140lbs=36lbs] 1 year of fat loss and 1 year of recomp. C'mon abs!,52
14404,"M/39/5'6"" [103kg &gt; 79kg = 24kg] 10 months of Keto FTW!",40


In [107]:
# since almost all the "days" entries are wrong, remove anything with period_weeks less than or equal to one week
pp_duration = pp_duration[pp_duration["period_weeks"] >= 1]

In [108]:
pp_duration.shape

(12623, 30)

In [109]:
pp_duration[["title", 'period_weeks']].sort_values("period_weeks", ascending = True)

,title,period_weeks
25762,M/17/5’10” [192 &gt; 187 = 5 lbs lost] 1 week progress! So excited to be back on track!!,1
27243,"M/24/6'0 [423 &gt; 276 = 147lbs] started January 20th of this year, my secret has been keto and the gym 6-7 days a week",1
12199,"M/22/6'1"" [246 &gt; 240 = 6LBS] (1 week) I decided it was time to make a change. I started eating healthy and working out. I'm going to be making weekly posts of my progress and need this sub to hold me accountable. My biggest struggle is keeping up the willpower to stick with being healthy.",1
8443,"M/44/5’6”/177&gt;173=4 (1week) Please be kind with critiques, I posted in a similar sub, hoping for more feedback and encouragement. Pics are censored, but still NSFW.",1
24268,"F/20/5'3"" [204.80 lbs&gt; 201.70 lbs] (1 week) my starting weight was 211.60 lbs and decided to take pics of my progress once a week",1
...,...,...
9862,"M/32/6'4"" [320 &gt; 200 = 120lbs ] (32 years) Healed my body and mind from childhood trauma.",1664
25183,M/39/5'11 [187lbs &gt;165lbs] (three Years) From newly divorced to present day. Found kettlebells and running. At least decent shape for a soon to be 40 years old.,2080
22671,F/49/5’10”[288lbs&gt;200lbs=88lbs] Today I turned 49 years old and I’ve never felt better (7 years),2548
3853,F / 29 / 5’3” (204 &gt; 189 = 15) still have 25lbs to go but I’m liking what I see. Started at 264 2 years ago,137384


In [110]:
pp_duration[["title", 'period_weeks']].sort_values("period_weeks", ascending = False).head(30)

,title,period_weeks
7412,M/23/5”10” [320lbs &gt; 185lbs = 135lbs lost] 2012 - 2018 6 years of every day gym!,1.04967e+06
3853,F / 29 / 5’3” (204 &gt; 189 = 15) still have 25lbs to go but I’m liking what I see. Started at 264 2 years ago,137384
22671,F/49/5’10”[288lbs&gt;200lbs=88lbs] Today I turned 49 years old and I’ve never felt better (7 years),2548
25183,M/39/5'11 [187lbs &gt;165lbs] (three Years) From newly divorced to present day. Found kettlebells and running. At least decent shape for a soon to be 40 years old.,2080
9862,"M/32/6'4"" [320 &gt; 200 = 120lbs ] (32 years) Healed my body and mind from childhood trauma.",1664
11989,"M/44/5’7” [185&gt;130=55 pounds lost] I’ve been very self conscience about my weight for almost 30 years, especially of my chest area. I wore a tight-fitting undershirt constantly. I’m not a tall man, so in my opinion, that made it worse. I decided to finally do something about it.",1560
24367,"M/30/5’7” [195lbs &gt; 145lbs =50lbs] today I turned 30 years old, feel better now than I ever did in my 20s.",1560
18918,Male / 29 / 5'11 [115lbs &gt; 205lbs = 95lbs] 12+ Years Natural Transformation: From Skinny to Aesthetics (17 to 29 years old | Motivational),1508
18908,Male / 29 / 5'11 [115 &gt; 205 =95] 12+ Years Natural Transformation: From Skinny to Aesthetics (17 to 29 years old | Motivational),1508
9227,"F/28/5’5 [210&gt;170=40] 1st pic I was 20 (my biggest) now I’m 28 years old a very happy, still 25 pounds to go!",1456


In [111]:
# a lot of the higher year entries are actually mis-interpretted ages.  Drop all durations greater than 850 weeks
pp_duration = pp_duration[pp_duration["period_weeks"] <= 850]

In [112]:
pp_duration[["title", 'period_weeks']].sort_values("period_weeks", ascending = False).head(30)

,title,period_weeks
8722,M/32/6'2 [340&gt;220&lt;280&gt;170&lt;240&gt;180 = 160 pounds] (16 years). Focused on no more yo-yo'ing.,832
5279,M/31/5’8 [225 &gt; 169 = 56] (204 months) high school me vs adult me.,816
4221,m/31/6'0 [250 lbs &gt; 170 lbs = 80 lbs] 15 years. It's been a long road y'all.,780
22339,"M/37/5'11"" [195 &gt; 160 = 35lbs](15 years) - Found an old photo at my parent's house. Long journey, still working harder.",780
24631,"M/36/6'1"" [240 &gt; 185 = 65](15 years). I got tired of hearing it's impossible to lose weight and keep it off, ignore the morning hair!",780
18319,M/36/6’2 (289&gt;199 = 90lbs of weight loss) finally broke the sub 200 lbs mark for the first time in 15 years.,780
4725,M/29/5’11” [200 lb to 178 lb] (15 years) + puberty,780
22420,"M/26/5'9"" [241lbs&gt; 189lbs = 52lbs] My major goal was to be below 190 lbs for the first time in 15 years. Next goal is to gain 20 lbs of muscle. NSFW",780
6488,F/28/5'6 [222lbs&gt;211.8 lbs] (oct/17-march/18 5 months) Started a new powerlifting program and kind of (but not really) cleaned up my nutrition/tracking my macros more accurately,740
23348,M/18/6’6” [ 160 &gt; 205 = 45 lbs gained ] My journey from being an insecure bulemic 14 year old to being genuinely happy with myself inside and out. This is what fitness is all about.,728


In [113]:
pp_duration.shape

(12595, 30)

In [114]:
# still some misinterpreation, but not too bad...

## Rate of weight loss

In [115]:
# calculate rate of weight loss in lbs/week, store in rate column
pp_duration["rate"] = pp_duration['weight_diff']/pp_duration["period_months"]

In [116]:
# calculate normalized rate of weight loss in lbs/week, store in norm_rate column
pp_duration['norm_rate'] = pp_duration["weight_diff"]/pp_duration["start_weight"]/pp_duration["period_months"]

In [117]:
# this revealed all sorts of problems with the duration caluculation...lots of .5 and 1/2 being misinterpretted leading 
# to crazy rates of weight loss.  
pp_duration.sort_values("rate", ascending=False).head(50)

,title,age,sex,height_in,start_weight,end_weight,weight_diff,period_weeks,period_months,NSFW,...,date,time,raw_sex,raw_height,num_height,raw_weights,raw_start_weight,raw_end_weight,rate,norm_rate
27243,"M/24/6'0 [423 &gt; 276 = 147lbs] started January 20th of this year, my secret has been keto and the gym 6-7 days a week",24.0,0,72.0,423.0,276.0,147.0,1,0.233333,0.0,...,2018-12-08,09:01:51,M,6'0,60,"423 &GT; 276 = 147LBS] STARTED JANUARY 20TH OF THIS YEAR, MY SECRET HAS BEEN KETO AND THE GYM 6-7 DAYS A WEEK",423.0,276.0,630,1.48936
17758,"F/44/5'4""[256&gt;147=101lbs](26mos) My momma (pictured) is my hero. Technically 26mos, 9days. Today marks 6 years of maintaining. (Currently around 155) She's struggled with her weight my whole life and its amazing to see her so happy with it now.",44.0,1,64.0,256.0,147.0,109.0,1.28571,0.3,0.0,...,2018-08-10,19:41:24,F,"5'4""",54,"256&GT;147=101LBS](26MOS) MY MOMMA (PICTURED) IS MY HERO. TECHNICALLY 26MOS, 9DAYS. TODAY MARKS 6 YEARS OF MAINTAINING. (CURRENTLY AROUND 155) SHE'S STRUGGLED WITH HER WEIGHT MY WHOLE LIFE AND ITS AMAZING TO SEE HER SO HAPPY WITH IT NOW.",256.0,147.0,363.333,1.41927
24741,"M/37/5'9""[360lbs&gt;198lbs=162lbs] (10.5 months) 9.5 months keto and IF, last 35 days at gym 1 hour a day. picture on left about 2007. Extremely hard to find before picture with no shirt. lol.",37.0,0,69.0,360.0,198.0,162.0,2,0.5,0.0,...,2018-11-07,01:39:07,M,"5'9""",59,"360LBS&GT;198LBS=162LBS] (10.5 MONTHS) 9.5 MONTHS KETO AND IF, LAST 35 DAYS AT GYM 1 HOUR A DAY. PICTURE ON LEFT ABOUT 2007. EXTREMELY HARD TO FIND BEFORE PICTURE WITH NO SHIRT. LOL.",360.0,198.0,324,0.9
24076,"M/24/5’10” [296lbs &gt; 165 lbs = 131lbs] (10.5 months) It’s been a beautiful journey of self-love and progress, and I’m excited to see where it’ll take me!",24.0,0,70.0,296.0,165.0,131.0,2,0.5,0.0,...,2018-11-04,15:32:12,M,5’10”,510,"296LBS &GT; 165 LBS = 131LBS] (10.5 MONTHS) IT’S BEEN A BEAUTIFUL JOURNEY OF SELF-LOVE AND PROGRESS, AND I’M EXCITED TO SEE WHERE IT’LL TAKE ME!",296.0,165.0,262,0.885135
865,M/43/6’5” [475&gt;345=130lbs down] 10.5 months,43.0,0,77.0,475.0,345.0,130.0,2,0.5,0.0,...,2018-01-23,19:09:09,M,6’5”,65,475&GT;345=130LBS DOWN] 10.5 MONTHS,475.0,345.0,260,0.547368
824,M/43/6’5” [475&gt;345=130] (10.5 months) Keto &amp; IF,43.0,0,77.0,475.0,345.0,130.0,2,0.5,0.0,...,2018-01-23,19:19:48,M,6’5”,65,475&GT;345=130] (10.5 MONTHS) KETO &AMP; IF,475.0,345.0,260,0.547368
25292,"F/26/5’7” [296lbs &gt; 200.6lbs = 95.4lbs] (10.5 months) So, so close to onederland. And so close to 100lbs lost. I won’t let Christmas beat me! Before picture from Christmas, 2016.",26.0,1,67.0,296.0,200.0,96.0,2,0.5,0.0,...,2018-12-10,14:33:28,F,5’7”,57,"296LBS &GT; 200.6LBS = 95.4LBS] (10.5 MONTHS) SO, SO CLOSE TO ONEDERLAND. AND SO CLOSE TO 100LBS LOST. I WON’T LET CHRISTMAS BEAT ME! BEFORE PICTURE FROM CHRISTMAS, 2016.",296.0,200.0,192,0.648649
12259,"F/31/5'7"" [246&gt;151=95lbs] 10.5 months all through counting calories and protein and consistency of exercise and a love for running",31.0,1,67.0,246.0,151.0,95.0,2,0.5,0.0,...,2018-06-12,05:04:14,F,"5'7""",57,246&GT;151=95LBS] 10.5 MONTHS ALL THROUGH COUNTING CALORIES AND PROTEIN AND CONSISTENCY OF EXERCISE AND A LOVE FOR RUNNING,246.0,151.0,190,0.772358
25332,"F/42/5’9” [230lbs &gt; 150lbs = 80lbs] (10.5 months) Headed into 2019 a new woman. From 34.0 - 22.1 BMI, 44% - 25% bodyfat, size 18-20 jeans to 6-8 jeans.",42.0,1,69.0,230.0,150.0,80.0,2,0.5,0.0,...,2018-12-04,14:58:04,F,5’9”,59,"230LBS &GT; 150LBS = 80LBS] (10.5 MONTHS) HEADED INTO 2019 A NEW WOMAN. FROM 34.0 - 22.1 BMI, 44% - 25% BODYFAT, SIZE 18-20 JEANS TO 6-8 JEANS.",230.0,150.0,160,0.695652
26962,"F/27/5'8"" [230 &gt; 170 &gt; 150 = 80lbs lost] (10.5 months) Side profile progress! Still a little more to go",27.0,1,68.0,230.0,170.0,60.0,2,0.5,0.0,...,2018-12-01,07:11:30,F,"5'8""",58,230 &GT; 170 &GT; 150 = 80LBS LOST] (10.5 MONTHS) SIDE PROFILE PROGRESS! STILL A LITTLE MORE TO GO,230.0,170.0,120,0.521739


In [118]:
# eliminate all weight loss rates greater than 30 lbs/month  or  less than -29 lbs/month.
pp_duration = pp_duration[(pp_duration["rate"] < 30) & (pp_duration["rate"] > -30)]

In [119]:
pp_duration.shape

(12517, 32)

## Save processed dataframe to a cvs file

In [120]:
# removing all starting and intermediate columns so that only processed features remain.
pp_duration_pro = pp_duration.reindex(columns=['age', 'sex', 'height_in', 'start_weight', 'end_weight', 'weight_diff', 'score', 
                                               'num_comments', 'month', 'dayofweek', 'NSFW', "num_posts", 
                                               "period_months", 'rate', 'norm_rate'])

In [121]:
# save df to file
pp_duration_pro.to_csv("pp_duration_2018_processed.csv", index=False)